https://towardsdatascience.com/introducing-geneal-a-genetic-algorithm-python-library-db69abfc212c

In [1]:
from Qsun.Qcircuit import *
from Qsun.Qgates import *
from Qsun.Qmeas import *
import numpy as np

In [2]:
def initialize_population(pop_size, n_genes, input_limits):

    population = np.random.randint(input_limits, size=(pop_size, n_genes))

    return population

In [3]:
def decoding_circuit(circuit, encoding_list, individual, params=None):
    
    n_qubit = int(len(individual)/5)
#     index_params = 0
    
    for i in range(n_qubit):
        for j in range(5):
            index = i*5+j
            if encoding_list[individual[index]] == 'H':
                H(circuit, i)
            elif encoding_list[individual[index]] == 'X':
                X(circuit, i)
            elif encoding_list[individual[index]] == 'Y':
                Y(circuit, i)
            elif encoding_list[individual[index]] == 'Z':
                Z(circuit, i)
            elif encoding_list[individual[index]] == 'S':
                S(circuit, i)
            elif encoding_list[individual[index]] == 'T':
                T(circuit, i)
            elif encoding_list[individual[index]] == 'Xsquare':
                Xsquare(circuit, i)
            elif encoding_list[individual[index]] == 'RX':
#                 RX(circuit, i, params[index_params])
#                 index_params += 1
                RX(circuit, i)
            elif encoding_list[individual[index]] == 'RY':
#                 RY(circuit, i, params[index_params])
#                 index_params += 1
                RY(circuit, i)
            elif encoding_list[individual[index]] == 'RZ':
#                 RZ(circuit, i, params[index_params])
#                 index_params += 1
                RZ(circuit, i)
    
    for i in range(n_qubit-1):
        CNOT(circuit, i, i+1)
    CNOT(circuit, n_qubit-1, 0)
    
    return circuit

def circuit_layers(n_layers, encoding_list, individual, params=None):
    
    circuit = Qubit(int(len(individual)/5))
    for i in range(n_layers):
#         circuit = decoding_circuit(circuit, encoding_list, individual, params[i])
        circuit = decoding_circuit(circuit, encoding_list, individual)
    return circuit    

In [4]:
def fitness_function(n_layers, encoding_list, individual):
    
    n_qubit = int(len(individual)/5)
    circuit = circuit_layers(n_layers, encoding_list, individual)
    
    expectation = np.array([measure_one(circuit, i)[1] for i in range(n_qubit)])
    return np.sum(expectation) - np.sum(individual < 10) / len(individual)

def calculate_fitness(population, n_layers, encoding_list):
    
    return np.array([fitness_function(n_layers, encoding_list, individual) for individual in population])

In [5]:
def select_parents(selection_strategy, n_matings, fitness, prob_intervals):
    """
    Selects the parents according to a given selection strategy.
    Options are:
    roulette_wheel: Selects individuals from mating pool giving
    higher probabilities to fitter individuals.
    
    :param selection_strategy: the strategy to use for selecting parents
    :param n_matings: the number of matings to perform
    :param prob_intervals: the selection probability for each individual in 
     the mating pool.
    :return: 2 arrays with selected individuals corresponding to each parent
    """

    ma, pa = None, None

    if selection_strategy == "roulette_wheel":

        ma = np.apply_along_axis(lambda value: np.argmin(value > prob_intervals) - 1, 1, np.random.rand(n_matings, 1))
        pa = np.apply_along_axis(lambda value: np.argmin(value > prob_intervals) - 1, 1, np.random.rand(n_matings, 1))
        
    return ma, pa

#     parents = np.random.choice(len(population), 2, p=fitness/sum(fitness), replace=False)
#     return parents[0], parents[1]

In [6]:
def create_offspring(first_parent, sec_parent, crossover_pt=None, offspring_number = "first"):
    
    if crossover_pt == None:
        crossover_pt = int(len(first_parent)/2)
    
    if offspring_number == "first":
        return np.hstack((first_parent[:crossover_pt], sec_parent[crossover_pt:]))
    
    elif offspring_number == "second":
        return np.hstack((sec_parent[:crossover_pt], first_parent[crossover_pt:]))

In [7]:
def mutate_population(population, n_mutations, input_limits):

    mutation_rows = np.random.choice(np.arange(1, population.shape[0]), n_mutations, replace=True)

    mutation_columns = np.random.choice(population.shape[1], n_mutations, replace=True)
    
    new_population = np.random.randint(input_limits, size=population.shape)

    population[mutation_rows, mutation_columns] = new_population[mutation_rows, mutation_columns]

    return population

In [8]:
def sort_by_fitness(fitness, population):
    
    sorted_fitness = np.argsort(fitness)[::-1]

    population = population[sorted_fitness, :]
    fitness = fitness[sorted_fitness]

    return fitness, population

In [9]:
def get_selection_probabilities(selection_strategy, pop_keep):

    if selection_strategy == "roulette_wheel":

        mating_prob = (np.arange(1, pop_keep + 1) / np.arange(1, pop_keep + 1).sum())[::-1]

        return np.array([0, *np.cumsum(mating_prob[: pop_keep + 1])])

    elif selection_strategy == "random":
        return np.linspace(0, 1, pop_keep + 1)

In [17]:
selection_strategy = "roulette_wheel" 
selection_rate = 0.5
mutation_rate = 0.1

n_genes = 20 # number of variables
pop_size = 40 # population size
pop_keep = math.floor(selection_rate * pop_size) # number of individuals to keep on each iteration

n_matings = math.floor((pop_size - pop_keep) / 2) # number of crossovers to perform
n_mutations = math.ceil((pop_size - 1) * n_genes * mutation_rate) # number o mutations to perform

# probability intervals, needed for roulete_wheel and random selection strategies
prob_intervals = get_selection_probabilities(selection_strategy, pop_keep)

max_gen = 100 # Maximum number of generations

encoding_list = {0: 'H',
                 1: 'X',
                 2: 'Y',
                 3: 'Z',
                 4: 'S',
                 5: 'T',
                 6: 'Xsquare',
                 7: 'RX',
                 8: 'RY',
                 9: 'RZ',
                 10: None}

input_limits = len(encoding_list)
n_layers = 2

In [18]:
population = initialize_population(pop_size, n_genes, input_limits)
population

array([[ 2, 10, 10,  8,  0,  1,  5,  6,  7,  1,  4,  8,  4,  3,  7,  1,
         3,  3,  9,  0],
       [ 5,  2,  7,  4,  9,  5,  8,  1,  6,  8, 10,  5,  0,  0, 10, 10,
         3, 10, 10, 10],
       [ 7,  8,  9,  3,  8,  0,  9,  4,  8,  3, 10,  0,  8, 10,  5,  3,
         9,  3,  7,  3],
       [ 6, 10,  2,  1,  8,  0,  5,  4,  3, 10,  8,  4,  0,  3,  5,  4,
         2, 10,  4,  6],
       [ 2,  9,  6,  1,  8,  2, 10,  5,  1,  6,  1,  8,  0, 10,  3,  9,
         5,  9,  2,  8],
       [ 3,  9,  4,  6, 10,  1, 10,  6,  3,  1,  8,  6,  1,  9,  3,  9,
         2,  8,  7,  5],
       [ 2,  6,  0,  1,  8,  1,  7,  5, 10,  3,  3,  9,  9,  8,  8,  6,
         2,  9,  7,  7],
       [10,  6,  0, 10,  5,  7,  6,  5,  4,  7, 10,  9,  0,  1,  8, 10,
         3,  2,  4,  7],
       [ 8,  9,  2,  2,  8,  2,  9,  8,  0,  1,  1,  2,  5, 10,  9,  6,
         7,  5,  8,  3],
       [ 7,  1, 10,  9,  4, 10,  5,  3,  4,  4,  4,  0,  7,  5,  7,  5,
         0,  0,  7, 10],
       [ 5,  9,  7,  1,  8,  2

In [19]:
circuit = circuit_layers(n_layers, encoding_list, population[0])
circuit.visual_circuit()

|Q_0> : Y--RY-H-----------------------------------------------o--------x--Y--RY-H-----------------------------------------------o--------x---M
                                                              |        |                                                        |        |    
|Q_1> : ---------X--T--XS--RX-X--------------------------------x--o-----------------X--T--XS--RX-X--------------------------------x--o---------M
                                                                 |     |                                                           |     |    
|Q_2> : ------------------------S--RY-S--Z--RX-------------------x--o-----------------------------S--RY-S--Z--RX-------------------x--o------M
                                                                    |  |                                                              |  |    
|Q_3> : ---------------------------------------X--Z--Z--RZ-H--------x--o-----------------------------------------X--Z--Z--RZ-H--------x--o--

In [20]:
# initialize the population
population = initialize_population(pop_size, n_genes, input_limits)

# Calculate the fitness of the population
fitness = calculate_fitness(population, n_layers, encoding_list)

# Sort population by fitness
fitness, population = sort_by_fitness(fitness, population)

fitness

array([2.2       , 1.45355339, 1.35355339, 1.35      , 1.25      ,
       1.25      , 1.25      , 1.2267767 , 1.2       , 1.2       ,
       1.15      , 1.15      , 1.15      , 1.15      , 1.15      ,
       1.15      , 1.15      , 1.1       , 1.1       , 1.1       ,
       1.1       , 1.1       , 1.1       , 1.1       , 1.05      ,
       1.05      , 1.05      , 1.05      , 1.05      , 1.05      ,
       1.05      , 1.05      , 1.05      , 1.05      , 1.        ,
       1.        , 0.89644661, 0.84644661, 0.69644661, 0.1       ])

In [21]:
gen_n = 0

while gen_n <= 100:

    gen_n += 1

    # Get parents pairs
    ma, pa = select_parents(selection_strategy, n_matings, fitness, prob_intervals)

    # Get indices of individuals to be replaced
    ix = np.arange(0, pop_size - pop_keep - 1, 2)

    for i in range(len(ix)):

        # create first offspring
        population[-1-ix[i]] = create_offspring(population[ma[i]], population[pa[i]], None, "first")

        # create second offspring
        population[-1-ix[i]-1] = create_offspring(population[ma[i]], population[pa[i]], None, "second")

    population = mutate_population(population, n_mutations, input_limits)
    
    fitness = calculate_fitness(population, n_layers, encoding_list)

    fitness, population = sort_by_fitness(fitness, population)

In [22]:
population[0], fitness[0]

(array([ 7,  2,  7, 10, 10, 10, 10, 10,  1, 10, 10,  5, 10, 10,  7, 10, 10,
        10, 10, 10]),
 3.7)

In [23]:
fitness

array([ 3.7 ,  3.7 ,  3.65,  3.6 ,  3.6 ,  3.6 ,  3.6 ,  3.6 ,  3.6 ,
        3.6 ,  3.55,  3.55,  3.55,  3.55,  3.55,  3.5 ,  3.45,  2.6 ,
        2.55,  2.5 ,  2.45,  1.65,  1.6 ,  1.6 ,  1.55,  1.55,  1.55,
        1.5 ,  1.5 ,  1.5 ,  1.5 ,  1.45,  1.35,  0.65,  0.55,  0.55,
        0.5 ,  0.5 ,  0.45, -0.5 ])

In [24]:
circuit = circuit_layers(n_layers, encoding_list, population[0])
circuit.visual_circuit()

|Q_0> : RX-Y--RX----------o--------x--RX-Y--RX----------o--------x---M
                          |        |                    |        |    
|Q_1> : ---------X--------x--o-----------------X--------x--o---------M
                             |     |                       |     |    
|Q_2> : ------------T--RX----x--o-----------------T--RX----x--o------M
                                |  |                          |  |    
|Q_3> : ------------------------x--o--------------------------x--o---M
                                                                      


In [16]:
expectation = np.array([measure_one(circuit, i)[1] for i in range(4)])
np.sum(expectation)

4.0